In [5]:
#!pip install google-cloud-bigquery pandas db-dtypes google-cloud-bigquery-storage
import os
from google.cloud import bigquery
import pandas as pd

import db_dtypes
print("db-dtypes is installed correctly.")


# Set the credentials for Google BigQuery
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "st312-442610-b9934d6c7287.json"
client = bigquery.Client()


db-dtypes is installed correctly.


### test Query - this is a small test query to see if everything is working

In [8]:
query = """
SELECT number FROM `bigquery-public-data.crypto_polygon.blocks` WHERE TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-02-06") LIMIT 1000
"""


# Run query and load into Pandas DataFrame
df = client.query(query).to_dataframe()

# Display results
df

,number
0,53210036
1,53211118
2,53173056
3,53210636
4,53176758
...,...
995,53210881
996,53176115
997,53171654
998,53173125
